In [0]:

import numpy as np
from tensorflow.python.client import device_lib
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout,Embedding
from keras.models import load_model
import string 
import tensorflow as tf
import math
from collections import OrderedDict
import collections
import random
import sys
from google.colab import files
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model



Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig libann0 libcairo2 libcdt5 libcgraph6 libdatrie1 libgd3
  libgts-0.7-5 libgts-bin libgvc6 libgvpr2 libjbig0 liblab-gamut1 libltdl7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4
  libpixman-1-0 libthai-data libthai0 libtiff5 libwebp6 libxaw7 libxcb-render0
  libxcb-shm0 libxmu6 libxpm4 libxt6
Suggested packages:
  gsfonts graphviz-doc libgd-tools
The following NEW packages will be installed:
  fontconfig graphviz libann0 libcairo2 libcdt5 libcgraph6 libdatrie1 libgd3
  libgts-0.7-5 libgts-bin libgvc6 libgvpr2 libjbig0 liblab-gamut1 libltdl7
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4
  libpixman-1-0 libthai-data libthai0 libtiff5 libwebp6 libxaw7 libxcb-render0
  libxcb-shm0 libxmu6 libxpm4 libxt6
0 upgraded, 30 newly installed, 0 to remove and 7 not upgraded.
Need to get 4,154 kB of

ModuleNotFoundError: ignored

In [0]:
import urllib
import nltk
from bs4 import BeautifulSoup
file=open("twotowers.txt",'w')

url='http://ae-lib.org.ua/texts-c/tolkien__the_lord_of_the_rings_1__en.htm'
html = urllib.request.urlopen(url)
soup=BeautifulSoup(html)

for x in soup(['script','style']):
  x.extract()

text=soup.get_text()


In [0]:

text=text[1211:-203].replace('\n',' ')

file.write(text)

995093

In [0]:
inputdata=open('twotowers.txt').read().lower()
#remove punctuation and newlines, keep exclamation marks, question marks, periods because they could all signify end of sentence
translate_table = dict((ord(char), None) for char in string.punctuation.replace('.','').replace('!','').replace('?','').replace("'",'').replace(',','')) 
inputdata=inputdata.translate(translate_table).replace('\n',' ')

inputdata=inputdata.replace('\n',' ').replace('.',' . ').replace('!',' ! ').replace('?',' ? ').replace("'"," ' ").replace(',',' , ').split()
#remove digits
for word in inputdata:
  if word.isdigit():
    inputdata.remove(word)
#add spaces between punctuation so we can split by space and treat them as words of their own


print(len(inputdata))

foreword t
219024


In [0]:

t=Tokenizer(filters='',char_level=False)
t.fit_on_texts(inputdata)
#create word dict
word2id=t.word_index
id2word={v:k for k,v in word2id.items()}

vocab_size=len(t.word_index)
print(vocab_size)
#encode text
data=[]
for word in inputdata:
  data.append(word2id[word])
print(len(data))

9672
9672
219024


In [0]:

input_seq=[]
max_n=6 #want max n_gram sequence length to be 6, up to 5 predictors and 1 label
#need to pad all sequences to be of at least length max_n


for i in range(2,max_n+1):
    for j in range(0,len(data)-max_n+1):
      input_seq.append(data[j:j+i])


In [0]:
#pad varied length n_grams
input_sequences=pad_sequences(input_seq,maxlen=max_n,padding='pre')
predictors,labels=input_sequences[:,:-1],input_sequences[:,-1]


In [0]:
print(predictors.shape,labels.shape)

embed_size=128


(1095095, 5) (1095095,)


In [0]:

#the output for the embed layer will be a vector of our embed size
#https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
#define model
#https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/
model=Sequential()
model.add(Embedding(vocab_size+1,embed_size,input_length=max_n-1))
model.add(LSTM(700,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(700,return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(700))
model.add(Dropout(0.3))
model.add(Dense(vocab_size+1,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 128)            1238144   
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 700)            2321200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 700)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 700)            3922800   
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 700)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 700)               3922800   
_________________________________________________________________
dropout_4 (Dropout)          (None, 700)               0         
__________

In [0]:

plot_model(model, to_file='model_plot2.png', show_shapes=True, show_layer_names=True)


ImportError: ignored

In [0]:
filepath="3layer30.h5"
checkpoint=ModelCheckpoint(filepath,monitor='val_loss',verbose=0,save_weights_only=False)


In [0]:
#model=load_model('2layer100.h5')
model.fit(predictors,labels,epochs=20,batch_size=500,callbacks=[checkpoint],verbose=1)
model.save('3layer50.h5')



Epoch 1/20
1095095/1095095 [==============================] - 420s 383us/step - loss: 2.6321
Epoch 2/20
1095095/1095095 [==============================] - 420s 383us/step - loss: 2.6105
Epoch 3/20
1095095/1095095 [==============================] - 419s 383us/step - loss: 2.5921
Epoch 4/20
1095095/1095095 [==============================] - 419s 383us/step - loss: 2.5746
Epoch 5/20
1095095/1095095 [==============================] - 420s 383us/step - loss: 2.5564
Epoch 6/20
1095095/1095095 [==============================] - 420s 383us/step - loss: 2.5387
Epoch 7/20
1095095/1095095 [==============================] - 420s 383us/step - loss: 2.5227
Epoch 8/20
1095095/1095095 [==============================] - 420s 383us/step - loss: 2.5067
Epoch 9/20
1095095/1095095 [==============================] - 419s 383us/step - loss: 2.4920
Epoch 10/20
1095095/1095095 [==============================] - 419s 383us/step - loss: 2.4777
Epoch 11/20
1095095/1095095 [==============================] - 419s 3

In [0]:

files.download('3layer30.h5')


----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 33486, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

In [0]:
#model=load_model('2layer100.h5')
def generate_text(seed_text, next_words, max_sequence_len, model):
    seed_text=seed_text.split()
    for i in range(next_words):
      if i%25==0:
        sys.stdout.write('\n')
      pattern=[]
      for word in seed_text:
        pattern.append(word2id[word])
      pattern=pad_sequences([pattern],maxlen=max_sequence_len-1,dtype='int32',padding='pre',truncating='pre',value=0.0)
      prediction=model.predict(pattern,verbose=0)
      prediction=np.argmax(prediction)
      prediction=id2word[prediction]
      sys.stdout.write(prediction+' ')
      seed_text.append(prediction)

  
  

In [0]:
generate_text('legolas and the orc',300,max_n,model)


, and knew that they were making songs of lamentation for his fall , for they were obliged to put on the ring , whether 
they think . it was not an orc . the next day the country on either side , and the gate closed silently behind me 
, and suddenly he felt extremely hungry . the proposal was welcomed by all and they sat down on the upper steps , dim figures 
in the gloom . ‘don’t you know my name yet ? that’s the only answer . tell me , who are you , and what 
do you want , and where do they come from , i ' d want it , if you want to know , i will 
tell you the truth , i was asked to look out for hobbits of the shire , and for a moment he stood trembling in 
the air . ‘old man willow ? naught worse than that , eh ? that can soon be mended . i have known strong warriors 
of the big people , though he demanded three guesses . the authorities , it is true , differ whether this last question was a 
mere ' question ' and not a sound of hoofs in the lane . suddenly as they drew nearer a t

In [0]:
model1=load_model('2layer30.h5')
model2=load_model('2layer50.h5')
model3=load_model('2layer100.h5')
model4=load_model('2layer150.h5')




In [0]:
#30
generate_text('legolas and the orc',100,max_n,model1)


. he was very reluctant to start , and the tides of fate are flowing . ' i have not spoken of it to the 
sackvillebagginses ! ‘for their own thought had been heard . but it was not a bad system . actually in hobbiton and bywater every day 
in the shire , but it is not my fate to sit in peace , even in the hilly regions and the older villages , 
such as hobbiton or tuckborough , or in the chief township of the shire , michel delving on the white downs at the lithe , 

In [0]:
#50
generate_text('legolas and the orc',100,max_n,model2)


, hut the fastwaning moon would not be seen till late . gimli and frodo were at the rear , grim and silent , walked 
aragorn . the passage twisted round a few turns , and then prepared to go down for the last time . in front of them 
they saw the lower lands dotted with small clumps of trees that melted away in the south . the swiftflowing clouds lifted and melted away 
, and the elves deemed that evil was ended for ever , and it was barely midday , when the shadowy shapes of the explorers 

In [0]:
#100
generate_text('legolas and the orc',100,max_n,model3)


! ' he cried . ' it was no ordinary storm . it is the ill will of the wood pressing on them . so 
silent was it that the figure of the elvenlady cast a dim shadow on the ground . where have you been , my friend ? 
why weren ' t you there ? ' cried frodo . ' ah ! ' said sam gloomily . ' we ' ll just wait 
long enough for winter to come . ' ' do you think the black riders have anything to do with it with gandalf ' s 

In [0]:
#150
generate_text('legolas and the orc',100,max_n,model4)


, and just an enemy . he deserves death . ’ ‘deserves it ! i daresay he does . many that live deserve death . 
and some that die deserve life . can you give it to them ? then do not be too eager to deal out death in 
judgement . for even the very wise cannot see all ends . i have not much hope that gollum can be cured before he dies 
, but there is a portageway somewhere on the western shore , from northern waste to southern hill , through dragonlair and hidden door and 